In [1]:
!python --version

Python 3.12.10


#### Pydantic Basics:
Pydantic models offer a powerful and elegant solution for data validation and parsing in Python. By leveraging standard Python type annotations, Pydantic automatically validates your data at runtime, ensuring consistency and dramatically reducing bugs.

#### Why Pydantic? The Advantages at a Glance

Before we dive into code, let's understand why Pydantic has become an indispensable tool for many Python developers:

- **Runtime Type Checking**: Catches data errors early, preventing unexpected behavior and difficult-to-debug issues.

- **Automatic Data Parsing**: Converts raw input (like JSON or dictionaries) into validated Python objects.

- **Clear, Concise Definitions**: Models are defined using standard Python type hints, making them highly readable and maintainable.

- **Serialization**: Easily convert models back into dictionaries or JSON.

- **Documentation Generation**: Models can serve as self-documenting schemas for your APIs and data structures.

- **Integration**: Seamlessly integrates with popular frameworks like FastAPI.

### The Foundation: Creating Your First Pydantic Models
Pydantic models are Python classes that inherit from `pydantic.BaseModel`. You define the expected data fields as class attributes using standard Python type annotations.

#### Example-1: Basic example to represent hero data

In [7]:
from pydantic import BaseModel, Field
from typing import Optional

# Basic Model Definition
class HeroProfile(BaseModel):
    id: int
    name: str
    gender: str

# Creating an instance (validation happens automatically)    
try:
    hero1 = HeroProfile(id=1, name="Superman", gender="M")
    print(f"First Hero details: {hero1}")
    
except Exception as e:
    print(f"\nError creating invalid user: {e}")

First Hero details: id=1 name='Superman' gender='M'


In [8]:
# Attempting to create an invalid instance (will raise ValidationError)
try:
    hero1 = HeroProfile(id="one", name="Superman", gender="M")
    print(f"First Hero details: {hero1}")
    
except Exception as e:
    print(f"\nError creating invalid user: {e}")


Error creating invalid user: 1 validation error for HeroProfile
id
  Input should be a valid integer, unable to parse string as an integer [type=int_parsing, input_value='one', input_type=str]
    For further information visit https://errors.pydantic.dev/2.11/v/int_parsing


#### Example-2: Adding Optional Fields and Default Values
Not all fields are always required. Pydantic handles this gracefully using typing.Optional and default values.

In [11]:
from pydantic import BaseModel
from typing import Optional

# Basic Model Definition
class HeroProfile(BaseModel):
    id: int
    name: str
    gender: str
    description: Optional[str] = None # Optional field with a default of None
    in_action: bool = True           # Field with a default value

# Creating an instance (validation happens automatically)    
try:
    hero1 = HeroProfile(id=1, name="Superman", gender="M")
    print(f"First Hero details: {hero1}")
    
    hero2 = HeroProfile(id=2, name="Batman", gender="M", description="Hero of Gotham city")
    print(f"First Hero details: {hero2}")
    
except Exception as e:
    print(f"\nError creating invalid user: {e}")

First Hero details: id=1 name='Superman' gender='M' description=None in_action=True
First Hero details: id=2 name='Batman' gender='M' description='Hero of Gotham city' in_action=True


Explanation:

- description: Optional[str] = None: This field can be a str or None. If not provided during instantiation, it defaults to None.

- in_stock: bool = True: This field is a bool and defaults to True if not explicitly provided.

- Pydantic validates types even for optional fields when values are provided

#### Example-3: Nested Models and Complex Structures
Pydantic excels at defining complex, nested data structures.

In [12]:
from pydantic import BaseModel, HttpUrl # HttpUrl for URL validation
from typing import List, Dict

class Address(BaseModel):
    street: str
    city: str
    zip_code: str
    country: str

class Company(BaseModel):
    name: str
    website: HttpUrl # Pydantic validates this as a valid URL
    employees: List[str]
    headquarters: Address # Nested Pydantic model
    departments: Dict[str, List[str]] # Dictionary with string keys and list of strings values

# Creating an instance of the Company model
try:
    company_data = {
        "name": "Tony Starc",
        "website": "https://www.tonystarc.com",
        "employees": ["Alice", "Bob", "Charlie"],
        "headquarters": {
            "street": "123 Main St",
            "city": "Innoville",
            "zip_code": "90210",
            "country": "USA"
        },
        "departments": {
            "Engineering": ["Alice", "David"],
            "Marketing": ["Eve"]
        }
    }

    company = Company(**company_data) # Using ** to unpack dictionary
    print(f"\nCompany Data: {company.model_dump_json(indent=2)}")

    # Attempting to create an invalid URL
    # invalid_company = Company(
    #     name="Bad Site Co.",
    #     website="not-a-url", # This will fail
    #     employees=[],
    #     headquarters=company_data["headquarters"],
    #     departments={}
    # )

except Exception as e:
    print(f"\nError creating invalid company: {e}")
    # Output will show a validation error for 'website'


Company Data: {
  "name": "Tony Starc",
  "website": "https://www.tonystarc.com/",
  "employees": [
    "Alice",
    "Bob",
    "Charlie"
  ],
  "headquarters": {
    "street": "123 Main St",
    "city": "Innoville",
    "zip_code": "90210",
    "country": "USA"
  },
  "departments": {
    "Engineering": [
      "Alice",
      "David"
    ],
    "Marketing": [
      "Eve"
    ]
  }
}


Explanation:

- We define an `Address` model and then use it as a type annotation for `headquarters` within the Company model.

- `HttpUrl` is a Pydantic-specific type that provides robust URL validation.

- `List[str]` and `Dict[str, List[str]]` demonstrate how to define lists and dictionaries with specific item types.

#### Example-4: Customization and Constraints
The Field function in Pydantic enhances model fields beyond basic type hints by allowing you to specify validation rules, default values, aliases, and more. Here's a comprehensive tutorial with examples.

In [13]:
from pydantic import BaseModel,Field
class Item(BaseModel):
    name:str=Field(min_length=2,max_length=50)
    price:float= Field(gt=0,le=1000) #greater than 0, less than or equal to 1000
    quantity:int=Field(ge=0)

# Valid instance
item = Item(name="Book", price=10, quantity=10)

print(item)

name='Book' price=10.0 quantity=10


In [14]:
from pydantic import BaseModel, Field

class User(BaseModel):
    username: str = Field(..., description="Unique username for the user")
    age: int = Field(default=18, description="User age, defaults to 18")
    email: str = Field(default_factory=lambda: "user@example.com", description="Default email address")

# Examples
user1 = User(username="alice")
print(user1)  # username='alice' age=18 email='user@example.com'

user2 = User(username="bob", age=25, email="bob@domain.com")
print(user2)  # username='bob' age=25 email='bob@domain.com'

username='alice' age=18 email='user@example.com'
username='bob' age=25 email='bob@domain.com'


#### Key Concepts and Best Practices

- **BaseModel**: The base class for all Pydantic models.

- **Type Annotations**: Crucial for defining the expected data types for each field.

- **Runtime Validation**: Pydantic automatically validates data upon instance creation.

- **ValidationError**: The specific exception raised when validation fails. Always be prepared to catch it.

- **model_dump() / model_dump_json() (Pydantic v2+) or dict() / json() (Pydantic v1)**: Methods to convert a model instance back into a dictionary or JSON string, respectively.

- **Immutability** (by default): Model instances are immutable by default in Pydantic v2, which can be configured. In v1, they are mutable. Be aware of your Pydantic version.

- **Field Function**: For more advanced validation rules, aliases, and metadata (e.g., Field(..., min_length=5, max_length=20)). We will explore this in more advanced topics.